In [97]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense, Activation, Dropout
from keras.layers import Masking
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [98]:
# Datei lesen
file_path = './cmudict.txt'

# Listen für Wörter und Phoneme
words = []
phonemes = []

with open(file_path, 'r') as file:
    for line in file:
        # Trennen des Wortes und der Phoneme beim ersten Leerzeichen
        word, phoneme_string = line.strip().split(' ', 1)
        word = re.sub(r'\(.*?\)', '', word)
        words.append(word)
        phonemes.append(phoneme_string[1:])

 

In [99]:
# Tokenisierung der Wörter und Phoneme
word_encoder = LabelEncoder()
phoneme_encoder = LabelEncoder()

In [100]:
all_chars = list(set(''.join(words)))
all_phonemes = list(set(' '.join(phonemes).split()))

In [101]:
word_encoder.fit(all_chars)
phoneme_encoder.fit(all_phonemes)

encoded_words = [word_encoder.transform(list(word)) for word in words]
encoded_phonemes = [phoneme_encoder.transform(phoneme.split()) for phoneme in phonemes]

In [102]:
# Anzahl der verschiedenen Zeichen und Phoneme
num_chars = len(word_encoder.classes_)
num_phonemes = len(phoneme_encoder.classes_)

In [103]:
# Padding der Sequenzen
max_word_length = max(len(word) for word in encoded_words)
max_phoneme_length = max(len(phoneme) for phoneme in encoded_phonemes)

In [104]:
# Padding der Sequenzen
max_length = max(max_word_length, max_phoneme_length)


In [105]:
# Woerter mit Padding versehen
padded_words = pad_sequences(encoded_words, maxlen=max_length, padding='post')
padded_phonemes = pad_sequences(encoded_phonemes, maxlen=max_length, padding='post')

In [110]:
# Modellaufbau
def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_encoder.classes_), output_dim=64))
    # Erste LSTM-Schicht mit Dropout
    model.add(Bidirectional(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    
    # Zweite LSTM-Schicht mit Dropout
    model.add(Bidirectional(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    
    # Dense-Schicht mit L2-Regularisierung
    model.add(TimeDistributed(Dense(128, activation='relu', kernel_regularizer=l2(0.01))))
    
 
    # Ausgabe-Schicht
    model.add(TimeDistributed(Dense(len(phoneme_encoder.classes_))))
    model.add(Activation('softmax'))
    
    # Kompilierung des Modells mit angepasstem Optimizer
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=[masked_accuracy])
    
    return model

model = create_model()

# Modellzusammenfassung
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_21 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_22 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_14                  │ ?                           │     0 (unbuilt) │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_15                  │ ?                           │     0 (unbuilt) │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_8 (Activation)            │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [106]:
# Umwandeln der Labels für das Training aka verschachteln der werte in einzelne arrays
y = np.expand_dims(padded_phonemes, -1)

In [45]:
import tensorflow as tf
# Eager Execution aktivieren
tf.config.run_functions_eagerly(False)


In [107]:
def masked_accuracy(y_true, y_pred):
   # Konvertiere Vorhersagen in Klassen
    y_pred_class = tf.argmax(y_pred, axis=-1)
    
    # Entferne die letzte Dimension von y_true
    y_true = tf.squeeze(y_true, -1)
    
    # Maskiere die gepolsterten Werte (Annahmen: 0 ist der gepolsterte Wert)
    mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    
    # Korrekte Vorhersagen
    matches = tf.cast(tf.equal(y_true, tf.cast(y_pred_class, y_true.dtype)), dtype=tf.float32)
    
    # Anzahl der korrekten Vorhersagen (ohne Padding)
    masked_matches = tf.reduce_sum(matches * mask)
    
    # Anzahl der gültigen Datenpunkte (ohne Padding)
    masked_count = tf.reduce_sum(mask)
    
    # Berechnung der Genauigkeit
    return masked_matches / masked_count

In [ ]:
model.fit(padded_words, y, epochs=5, batch_size=32, validation_split=0.2)



Epoch 1/5
 342/3358 ━━━━━━━━━━━━━━━━━━━━ 6:01 120ms/step - loss: 1.6510 - masked_accuracy: 0.0457

In [30]:
# Modell speichern
model.save('word_to_phoneme_model_good.keras')



In [19]:
def predict_phonemes(word):
    word = re.sub(r'\(.*?\)', '', word)
    encoded_word = word_encoder.transform(list(word))
    padded_word = pad_sequences([encoded_word], max_length, padding="post")
    prediction = model.predict(padded_word)
    predicted_phonemes = np.argmax(prediction, axis=-1)
    return phoneme_encoder.inverse_transform(predicted_phonemes[0])

In [31]:
new_word = "SHORT"
predicted_phonemes = predict_phonemes(new_word)
print(" ".join(predicted_phonemes))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
SH AO1 R T AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0 AA0


In [31]:
model = tf.keras.models.load_model("word_to_phoneme_model_masking.keras")